# CUDA MODULE DOESN'T LOAD IN JUPYTER NOTEBOOKS
I don't know what the issue is. The CPU version of this code loads fine, but when I try to run the include statement for the CUDA module the code hangs.

Great news is, if you run the same code in a Python script everything works as expected

In [1]:
import os
import numpy as np
from juliacall import Main as jl

/home/crashoverride/miniforge3/envs/2024_05_work/lib/python3.12/site-packages/juliacall/__init__.py:237: UserWarning: Julia was started with multiple threads but multithreading support is experimental in JuliaCall. It is recommended to restart Python with the environment variable PYTHON_JULIACALL_HANDLE_SIGNALS=yes set, otherwise you may experience segfaults or other crashes. Note however that this interferes with Python's own signal handling, so for example Ctrl-C will not raise KeyboardInterrupt. See https://juliapy.github.io/PythonCall.jl/stable/faq/#Is-PythonCall/JuliaCall-thread-safe? for further information. You can suppress this warning by setting PYTHON_JULIACALL_HANDLE_SIGNALS=no.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


JuliaCall should auto detect IPython and activate an extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython for info on how to use it.

Even though I could use IPython magic commands, I'm not going to because ultimately I want this to run in a script.

In [2]:
# Get the currebt working directory of the script
# with `os.getcwd()`. Relative to this script
# the path to the julia functions is `../../julia_fns`
# so we can join the current working directory with
# the relative path, then use `os.path.normpath` to
# get the absolute path. We can then save this as a
# global variable in Julia to be used by other
# scripts.
julia_fns_path = os.path.normpath(os.path.join(os.getcwd(), '../../julia_fns'))
print(julia_fns_path)


/home/crashoverride/Dropbox/code/vorpy/julia_fns


In [3]:
str(julia_fns_path)

'/home/crashoverride/Dropbox/code/vorpy/julia_fns'

In [4]:
# Make the global variable `JULIA_FNS`, in Julia,
# to store the absolute path to `julia_fns`.
jl.seval(f'JULIA_FNS = "{julia_fns_path}"')
print('JULIA_FNS:', jl.JULIA_FNS)
print('Type of JULIA_FNS:', jl.typeof(jl.JULIA_FNS))

JULIA_FNS: /home/crashoverride/Dropbox/code/vorpy/julia_fns
Type of JULIA_FNS: String


In [5]:
# Next, `include` the file 'julia_env.jl' which sets
# several global variables used by the vorpy Julia
# functions.
jl.seval('include(string(JULIA_FNS, "/julia_env.jl"))')

Julia environment variables loaded for Vorpy.


In [6]:
# Finally, we activate Julia project where
# project dependencies are installed. The project toml
# and manifest are located in `julia_fns`.
# We, reuse the global variable `JULIA_FNS` here
# as a check on the `JULIA_FNS` variable. If it is not
# set correctly, the following command will fail.
jl.Pkg.activate(jl.JULIA_FNS)


  Activating project at `~/Dropbox/code/vorpy/julia_fns`


In [7]:
jl.Pkg.instantiate()

In [8]:
# We can make sure we have the correct project activated
# by checking the status of the project (i.e. print a list
# of installed packages and their versions).
jl.Pkg.status()

Status `~/Dropbox/code/vorpy/julia_fns/Project.toml`
⌃ [6e4b80f9] BenchmarkTools v1.4.0
⌃ [052768ef] CUDA v5.2.0
⌃ [de52edbc] Integrals v4.1.0
⌃ [91a5bcdd] Plots v1.39.0
⌃ [90137ffa] StaticArrays v1.9.2
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [9]:
# Initialize the _WBS_SOLVER_DEVICES dictionary.
# We will determine what devices are available
# to the user below.
_WBS_SOLVER_DEVICES = {}

In [ ]:
jl.seval('include(string(JULIA_FNS, "/weighted_biot_savart_solver_cuda.jl"))')

In [ ]:
# Try to load the CUDA Biot-Savart solver. If it works
# add it to the _WBS_SOLVER_DEVICES dictionary.

# print('''Trying to load Biot-Savart solver. If this is the first time
#       running this script, it may take several minutes to compile the
#       Julia dependencies. Note: during this time, no information is printed
#       to the terminal. Subsequent runs should be faster.''')

try:
    # Load CUDA Biot-Savart function
    jl.include(jl.JULIA_FNS + '/weighted_biot_savart_solver_cuda.jl')

    def _wbs_solver_cuda(fps, vpps, vcrds, vcirs):
        return np.transpose(jl.weighted_biot_savart_solver_cuda(np.transpose(fps),
                                                                np.transpose(vpps),
                                                                vcrds,
                                                                vcirs))
    
    _WBS_SOLVER_DEVICES['cuda'] = _wbs_solver_cuda

except:
    print('CUDA version of Biot-Savart solver not available; revert to CPU version.')

finally:
    # Load CPU Biot-Savart function.
    jl.include(jl.JULIA_FNS + '/weighted_biot_savart_solver_cpu.jl')

    # User API
    def _wbs_solver_cpu(fps, vpps, vcrds, vcirs):
        return np.transpose(jl.weighted_biot_savart_solver_cpu(np.transpose(fps),
                                                                np.transpose(vpps),
                                                                vcrds,
                                                                vcirs))
    
    _WBS_SOLVER_DEVICES['cpu'] = _wbs_solver_cpu

    # User API
    def wbs_solve(fps, vpps, vcrds, vcirs, device='cpu'):
        try:
            return _WBS_SOLVER_DEVICES[device](fps, vpps, vcrds, vcirs)
        except KeyError:
            raise ValueError(f'Invalid device: \'{device}\'. Available devices: {list(_WBS_SOLVER_DEVICES.keys())}')

